In [24]:
import numpy as np
import pandas as pd
import re
import glob
import os 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 

In [25]:
path_train="scores/stack_train/"
path_test="scores/stack_test/"

In [28]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub("\.csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub("\.csv","",f)] = pd.read_csv(path_test+f)

In [29]:
stack_train=dict_train["nowdw_XGB_medium"].merge(dict_train["nowdw_XGB_small"],
on="ID").merge(dict_train["nowdw_XGB_tall"],on="ID").merge(dict_train["XGB_M_d2v"],
on="ID").merge(dict_train["XGB_S_d2v"],on="ID").merge(dict_train["XGB_T_d2v"],
on="ID").merge(dict_train["XGB_S_tfidf"],on="ID").merge(dict_train["XGB_M_tfidf"],
on="ID").merge(dict_train["XGB_T_tfidf"],on="ID").merge(dict_train["XGB_S_w2v"],
on="ID").merge(dict_train["XGB_M_w2v"],on="ID").merge(dict_train["XGB_T_w2v"],
on="ID").merge(dict_train["LGBM_S_d2v"],on="ID").merge(dict_train["LGBM_M_d2v"],
on="ID").merge(dict_train["LGBM_T_d2v"],on="ID").merge(dict_train["LGBM_S_tfidf"],
on="ID").merge(dict_train["LGBM_M_tfidf"],on="ID").merge(dict_train["LGBM_T_tfidf"],
on="ID").merge(dict_train["LGBM_S_w2v"],on="ID").merge(dict_train["LGBM_M_w2v"],
on="ID").merge(dict_train["LGBM_T_w2v"],on="ID").merge(dict_train["adaboost_d2v"],
on="ID").merge(dict_train["adaboost_tfidf"],on="ID").merge(dict_train["adaboost_w2v"],
on="ID").merge(dict_train["logit_d2v"],on="ID").merge(dict_train["logit_tfidf"],
on="ID").merge(dict_train["logit_w2v"],on="ID").merge(dict_train["knn_d2v"],
on="ID").merge(dict_train["knn_tfidf"],on="ID").merge(dict_train["knn_w2v"],
                    on="ID").merge(dict_train["nowdw_neural_net"],on="ID")
#a tester 

In [30]:
stack_test=dict_test["nowdw_XGB_medium"].merge(dict_test["nowdw_XGB_small"],
on="ID").merge(dict_test["nowdw_XGB_tall"],on="ID").merge(dict_test["XGB_M_d2v"],
on="ID").merge(dict_test["XGB_S_d2v"],on="ID").merge(dict_test["XGB_T_d2v"],
on="ID").merge(dict_test["XGB_S_tfidf"],on="ID").merge(dict_test["XGB_M_tfidf"],
on="ID").merge(dict_test["XGB_T_tfidf"],on="ID").merge(dict_test["XGB_S_w2v"],
on="ID").merge(dict_test["XGB_M_w2v"],on="ID").merge(dict_test["XGB_T_w2v"],
on="ID").merge(dict_test["LGBM_S_d2v"],on="ID").merge(dict_test["LGBM_M_d2v"],
on="ID").merge(dict_test["LGBM_T_d2v"],on="ID").merge(dict_test["LGBM_S_tfidf"],
on="ID").merge(dict_test["LGBM_M_tfidf"],on="ID").merge(dict_test["LGBM_T_tfidf"],
on="ID").merge(dict_test["LGBM_S_w2v"],on="ID").merge(dict_test["LGBM_M_w2v"],
on="ID").merge(dict_test["LGBM_T_w2v"],on="ID").merge(dict_test["adaboost_d2v"],
on="ID").merge(dict_test["adaboost_tfidf"],on="ID").merge(dict_test["adaboost_w2v"],
on="ID").merge(dict_test["logit_d2v"],on="ID").merge(dict_test["logit_tfidf"],
on="ID").merge(dict_test["logit_w2v"],on="ID").merge(dict_test["knn_d2v"],
on="ID").merge(dict_test["knn_tfidf"],on="ID").merge(dict_test["knn_w2v"],
                    on="ID").merge(dict_test["nowdw_neural_net"],on="ID")

In [31]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [32]:
y=pd.read_csv("bases/training_variants").Class.values - 1

In [33]:
ID_test=stack_test["ID"]

In [34]:
X_train=np.array(stack_train.drop("ID",axis=1))

In [35]:
X_test=np.array(stack_test.drop("ID",axis=1))

In [44]:
param_test1 = {
 'max_depth':[1,3,5,7,9],
 'min_child_weight':[1,3,5,7,9]
}
param_test1_bis={
    'max_depth':[1,2,3,4,5,7,9],
 'min_child_weight':[1,3,5,7]
}
param_test2 = {
 'gamma':[i for i in range(1,10)]
}
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
param_test3_bis = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
param_test4 = {
 'reg_alpha':[0.1,0.3,0.5,0.7,0.9]
}
param_test4_bis = {
 'reg_alpha':[i for i in np.arange(94,96,.2)]
}
param_test5 = {
 "learning_rate" : [i for i in np.arange(0.05,0.15,0.025)]
}
param_test6 = {
    "n_estimators" : [i for i in range(50, 300, 50)]
}


In [39]:
gsearch1=GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,gamma=6,n_estimators=200),param_grid = param_test1,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y)
gsearch1.grid_scores_, gsearch1_bis.best_params_, gsearch1_bis.best_score_

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.37745, std: 0.04417, params: {'max_depth': 1, 'min_child_weight': 1},
  mean: -0.37737, std: 0.04418, params: {'max_depth': 1, 'min_child_weight': 3},
  mean: -0.37716, std: 0.04379, params: {'max_depth': 1, 'min_child_weight': 5},
  mean: -0.37620, std: 0.04315, params: {'max_depth': 1, 'min_child_weight': 7},
  mean: -0.37609, std: 0.04253, params: {'max_depth': 1, 'min_child_weight': 9},
  mean: -0.37100, std: 0.05525, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: -0.36960, std: 0.05471, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: -0.36913, std: 0.05437, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: -0.36812, std: 0.05433, params: {'max_depth': 3, 'min_child_weight': 7},
  mean: -0.36783, std: 0.05341, params: {'max_depth': 3, 'min_child_weight': 9},
  mean: -0.37462, std: 0.05413, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: -0.37071, std: 0.05471, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: -0.36922, std: 0.052

In [43]:
gsearch2 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=5,min_child_weight=9,gamma=4,n_estimators=200),param_grid = param_test2,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.37764, std: 0.07344, params: {'gamma': 1},
  mean: -0.36598, std: 0.06443, params: {'gamma': 2},
  mean: -0.35869, std: 0.05881, params: {'gamma': 3},
  mean: -0.35659, std: 0.05807, params: {'gamma': 4},
  mean: -0.35929, std: 0.05476, params: {'gamma': 5},
  mean: -0.36406, std: 0.05317, params: {'gamma': 6},
  mean: -0.36992, std: 0.05125, params: {'gamma': 7},
  mean: -0.37458, std: 0.05150, params: {'gamma': 8},
  mean: -0.37957, std: 0.05119, params: {'gamma': 9}],
 {'gamma': 4},
 -0.35659368456848306)

In [45]:
gsearch3 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=5,min_child_weight=9,gamma=4,n_estimators=200),param_grid = param_test3,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.36632, std: 0.06287, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: -0.36289, std: 0.06102, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: -0.36232, std: 0.05489, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: -0.36409, std: 0.05611, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: -0.36585, std: 0.06147, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: -0.35824, std: 0.05773, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: -0.35979, std: 0.06083, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: -0.36062, std: 0.05940, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: -0.36360, std: 0.06795, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: -0.35706, std: 0.05851, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: -0.35843, std: 0.06317, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: -0.35508, std: 0.05975, params: {'colsample_bytree': 0.8,

In [47]:
gsearch4 = GridSearchCV(estimator = XGBClassifier(n_estimators=200,objective="multi:softprob",
seed=26,max_depth=5,min_child_weight=9,gamma=4,colsample_bytree=0.8,subsample=0.9,reg_alpha=0.5),param_grid = param_test4,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.35570, std: 0.05928, params: {'reg_alpha': 0.1},
  mean: -0.35595, std: 0.06030, params: {'reg_alpha': 0.3},
  mean: -0.35475, std: 0.05805, params: {'reg_alpha': 0.5},
  mean: -0.35765, std: 0.05885, params: {'reg_alpha': 0.7},
  mean: -0.35595, std: 0.05873, params: {'reg_alpha': 0.9}],
 {'reg_alpha': 0.5},
 -0.35475223505015158)

In [48]:
gsearch5 = GridSearchCV(estimator = XGBClassifier(n_estimators=200,objective="multi:softprob",
seed=26,max_depth=5,min_child_weight=9,gamma=4,colsample_bytree=0.8,subsample=0.9,learning_rate=0.1,reg_alpha=0.5),param_grid = param_test5,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_train,y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.35774, std: 0.05822, params: {'learning_rate': 0.050000000000000003},
  mean: -0.35753, std: 0.05818, params: {'learning_rate': 0.075000000000000011},
  mean: -0.35508, std: 0.05975, params: {'learning_rate': 0.10000000000000002},
  mean: -0.35803, std: 0.06193, params: {'learning_rate': 0.12500000000000003}],
 {'learning_rate': 0.10000000000000002},
 -0.35508297195615335)

In [51]:
gsearch6 = GridSearchCV(estimator = XGBClassifier(n_estimators=200,objective="multi:softprob",
seed=26,max_depth=5,min_child_weight=9,gamma=4,colsample_bytree=0.8,subsample=0.9,learning_rate=0.1,reg_alpha=0.5),param_grid = param_test6,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,y)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

/Users/LF/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.37057, std: 0.05743, params: {'n_estimators': 50},
  mean: -0.35626, std: 0.05797, params: {'n_estimators': 100},
  mean: -0.35493, std: 0.05785, params: {'n_estimators': 150},
  mean: -0.35475, std: 0.05805, params: {'n_estimators': 200},
  mean: -0.35477, std: 0.05811, params: {'n_estimators': 250}],
 {'n_estimators': 200},
 -0.35475223505015158)

In [52]:
clf = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=5,min_child_weight=9,gamma=3,colsample_bytree=0.8,subsample=0.9,learning_rate=0.1,reg_alpha=0.5)
                     
clf.fit(X_train,y)
y_test=clf.predict_proba(X_test)

classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
sub = pd.DataFrame(y_test, columns=classes)
sub['ID'] = ID_test
sub.to_csv("scores/stacking_finish_grdsearch.csv",index=False)